In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../../")

In [ ]:
from tokenizers import Tokenizer
import sys

import matplotlib.pyplot as plt
import numpy as np
import collections
import torch

from ChEmbed.data import chembldb, datasets, chembed_tokenize
from ChEmbed.training import trainer
from ChEmbed.modules import simple_rnn
import attr

from ChEmbed import plots, utilities

from transformers.tokenization_utils_fast import PreTrainedTokenizerFast
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
chembl_raw = chembldb.ChemblDBChemreps()
chembl_smiles = chembl_raw._load_or_download()["canonical_smiles"].to_list()

In [ ]:
chembl = datasets.SMILESDatasetContinuous(
    smiles_list = chembl_smiles,
    tokenizer = tokenizer,
    length = 65,
)

In [ ]:
print(chembl_mini[0][0].shape)
print(chembl_mini[0][1].shape)

In [ ]:
# print(predict("C=O", 30, model, chembl_mini, device="cpu"))

In [ ]:
model = simple_rnn.simpleRNN(
    # Mandatory
    num_hiddens = 128,
    vocab_size = len(tokenizer),
    # tuning
    learning_rate = 0.01,
    weight_decay = 0.05
)

In [ ]:
batch = next(iter(chembl_mini.train_dataloader()))
print(batch[0].shape)
print(batch[1].shape)

# [batch_size, seq_len, vocab_size]
# [batch_size, seq_len]

In [ ]:
test_smiles = "CN=C=O"
smiles_one_hot = chembl_mini.encode_smiles_to_one_hot(test_smiles)

outputs = model(smiles_one_hot.unsqueeze(0))

In [ ]:
print(smiles_one_hot.shape)
print(outputs.shape)

In [ ]:
print(chembl_mini.tokenizer.decode(smiles_one_hot.argmax(1).tolist()))
print(chembl_mini.tokenizer.decode(outputs.squeeze().argmax(1).tolist()))

In [ ]:
generate_sequence(
    prefix="C=O",
    num_chars=30,
    model=model,
    dataset=chembl_mini,
    device="cpu",
    temperature=0.01
)

In [ ]:

# Test the generation function
print("Testing generation with trained model:")
result = simple_generate("C=O", 5, model, chembl_mini, device="cpu")
print(f"Final result: {result}")

In [ ]:
model_trainer = trainer.Trainer(max_epochs=32, init_random=None, clip_grads_norm=1.0)
model_trainer.fit(model, chembl_mini)

In [ ]:
generate_sequence(
    prefix="C=O",
    num_chars=30,
    model=model,
    dataset=chembl_mini,
    device="cuda",
    temperature=0.1
)

In [ ]:
generate_sequence(
    prefix="C1",
    num_chars=30,
    model=model,
    dataset=chembl_mini,
    device="cuda",
    temperature=0.1
)

In [ ]:
# print gradients of model
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.grad)

In [ ]:
losses = utilities.extract_training_losses(model_trainer.metadata)
fig, ax = plots.plot_training_validation_loss(losses['avg_train_losses'], losses['avg_val_losses'])
ax.set_yscale('log')

In [ ]:
encoded = chembl_mini.tokenizer.encode("C")
encoded_1h = chembl_mini.encoding_to_one_hot(encoded)
print(encoded)
print(encoded_1h.argmax())

In [ ]:
pred_raw = model(encoded_1h)


In [ ]:
pred_raw.shape

In [ ]:
tokens = pred_raw.argmax(1).tolist()

In [ ]:
print(chembl_mini.tokenizer.decode(tokens))

In [ ]:
chembl_mini.tokenizer.decode(chembl_mini[0][0].argmax(1).tolist())

In [ ]:
chembl_mini.tokenizer.decode(chembl_mini[0][1].argmax(0).tolist())